In [1]:


from rdflib import *

In [2]:
ontology = Graph()
ontology.parse(r'../ontologies/ontology_full.ttl')

dmop = Namespace('http://www.e-lico.eu/ontologies/dmo/DMOP/DMOP.owl#')
bigowl_data = Namespace('https://w3id.org/BIGOWLData/')
bigowl_algorithms = Namespace('https://w3id.org/BIGOWLAlgorithms/')
bigowl_problems = Namespace('https://w3id.org/BIGOWLProblems/')
bigowl_workflows = Namespace('https://w3id.org/BIGOWLWorkflows/')
do = Namespace('https://diviloper.dev/ontology#')
dc = Namespace('https://diviloper.dev/ontology/components#')
dr = Namespace('https://diviloper.dev/ontology/restrictions#')
dru = Namespace('https://diviloper.dev/ontology/rules#')
da = Namespace('https://diviloper.dev/ABox#')


def get_graph():
    g = Graph()
    g.bind('dmop', dmop)
    g.bind('bod', bigowl_data)
    g.bind('boa', bigowl_algorithms)
    g.bind('bop', bigowl_problems)
    g.bind('bow', bigowl_workflows)
    g.bind('do', do)
    g.bind('dc', dc)
    g.bind('dr', dr)
    g.bind('da', da)
    return g

In [48]:
penguins = get_graph()
penguins.parse(r'../dataset_annotator/penguins_annotated.ttl', format='turtle')

<Graph identifier=N9ec4ab166a3546e8ad69e3eca15b7659 (<class 'rdflib.graph.Graph'>)>

In [10]:
dataset_id = penguins.value(predicate=RDF.type, object=do.TabularDataset)
dataset_id

rdflib.term.URIRef('https://diviloper.dev/ABox#penguins.csv')

In [39]:
penguins.query(f"""
PREFIX bod: <{bigowl_data}>
PREFIX div: <{do}>
SELECT ?s ?p2 ?c
 WHERE {{
    <{dataset_id}> bod:hasColumn ?c .
    ?c div:containsNulls true .
    ?c ?p ?o .
    ?s ?p2 ?c .
}}
""").bindings

[]

In [49]:
drop_null_columns_query = """
PREFIX bod: <https://w3id.org/BIGOWLData/>
PREFIX div: <https://diviloper.dev/ontology#>
DELETE
{
    ?c ?p ?o .
    ?s ?p2 ?c .
}
WHERE
{
    <$output1> bod:hasColumn ?c .
    ?c div:containsNulls true .
    ?c ?p ?o .
    ?s ?p2 ?c .
}
"""

In [ ]:
drop_null_rows_query = f"""
PREFIX bod: <{bigowl_data}>
PREFIX div: <{do}>
INSERT
{{
    ?c div:containsNulls false .
}}
DELETE
{{
    ?c div:containsNulls true .
}}
WHERE
{{
    <{dataset_id}> bod:hasColumn ?c .
    ?c div:containsNulls true .
}}
"""

In [ ]:
drop_null_column_query = """
PREFIX bod: <https://w3id.org/BIGOWLData/>
PREFIX div: <https://diviloper.dev/ontology#>
DELETE {
    ?c ?p ?o .
    ?s ?p2 ?c .
}
WHERE {
    <$output1> bod:hasColumn ?c .
    ?c bod:hasName "$parameter1".
    ?c ?p ?o .
    ?s ?p2 ?c .
}
"""

In [50]:
penguins.update(drop_null_columns_query)

In [51]:
penguins.serialize(f'penguins_drop_null_columns.ttl')

<Graph identifier=N9ec4ab166a3546e8ad69e3eca15b7659 (<class 'rdflib.graph.Graph'>)>